In [1]:
!pip install -Uqq fastbook

In [2]:
import fastbook
fastbook.setup_book()
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])


In [5]:
txt = files[0].open().read(); txt[:75]

'Jiang Xian uses the complex backstory of Ling Ling and Mao Daobing to study'

In [6]:
word_tokenizer = WordTokenizer()
toks = first(word_tokenizer([txt]))
print(coll_repr(toks, 30))

(#143) ['Jiang','Xian','uses','the','complex','backstory','of','Ling','Ling','and','Mao','Daobing','to','study','Mao',"'s",'"','cultural','revolution','"','(','1966','-','1976',')','at','the','village','level','.'...]


In [7]:
first(word_tokenizer(['The U.S. dollar 1.00.']))

(#5) ['The','U.S.','dollar','1.00','.']

In [8]:
tkn = Tokenizer(word_tokenizer)
print(coll_repr(tkn(txt), 31))

(#158) ['xxbos','xxmaj','jiang','xxmaj','xian','uses','the','complex','backstory','of','xxmaj','ling','xxmaj','ling','and','xxmaj','mao','xxmaj','daobing','to','study','xxmaj','mao',"'s",'"','cultural','revolution','"','(','1966','-'...]


In [9]:
txts = L(o.open().read() for o in files[:2000])

In [10]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [11]:
subword(1000)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


'▁J i ang ▁ X ian ▁us es ▁the ▁comp le x ▁back st or y ▁of ▁L ing ▁L ing ▁and ▁Ma o ▁Da o b ing ▁to ▁st u d y ▁Ma o \' s ▁" c ul'

In [12]:
subword(3000)

'▁J ian g ▁ X ian ▁use s ▁the ▁complex ▁back st ory ▁of ▁L ing ▁L ing ▁and ▁Ma o ▁Da ob ing ▁to ▁study ▁Ma o \' s ▁" cul t ur al ▁ r evolution " ▁('

In [13]:
subword(200)

'▁ J i an g ▁ X i an ▁ us es ▁the ▁c o m p le x ▁b a ck st or y ▁of ▁ L ing ▁ L ing ▁and ▁ M a o ▁ D a'

Numericalization : Process of mapping tokens to integers - Remember! Deep learning neural networks play around with numbers (usually normalized). 

In [14]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#158) ['xxbos','xxmaj','jiang','xxmaj','xian','uses','the','complex','backstory','of','xxmaj','ling','xxmaj','ling','and','xxmaj','mao','xxmaj','daobing','to','study','xxmaj','mao',"'s",'"','cultural','revolution','"','(','1966','-'...]


In [15]:
type(tkn(txt))

fastcore.foundation.L

In [16]:
print(list(tkn(txt))[:31])

['xxbos', 'xxmaj', 'jiang', 'xxmaj', 'xian', 'uses', 'the', 'complex', 'backstory', 'of', 'xxmaj', 'ling', 'xxmaj', 'ling', 'and', 'xxmaj', 'mao', 'xxmaj', 'daobing', 'to', 'study', 'xxmaj', 'mao', "'s", '"', 'cultural', 'revolution', '"', '(', '1966', '-']


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]


(#158) ['xxbos','xxmaj','jiang','xxmaj','xian','uses','the','complex','backstory','of'...]

In [18]:
num = Numericalize(min_freq=1) #fastai method to numericalize
num.setup(toks200)
coll_repr(num.vocab,20)


"(#7584) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [19]:
len(num.vocab)

7584

The way numericalization is handled is that max_vocab is set to 60000 as a large number of words would require large embedding matrix and there will not be enough RAM and processing time. Also, min_freq = 3 -> for a word to be set , it should appear atleast 3 times.

In [20]:
print(toks[:20])

['xxbos', 'xxmaj', 'jiang', 'xxmaj', 'xian', 'uses', 'the', 'complex', 'backstory', 'of', 'xxmaj', 'ling', 'xxmaj', 'ling', 'and', 'xxmaj', 'mao', 'xxmaj', 'daobing', 'to']


In [21]:
nums = num(toks[:20]);
nums

TensorText([   2,    8, 3392,    8, 3393, 1269,    9, 1270, 3394,   14,    8, 2148,    8, 2148,   12,    8, 2149,    8, 3395,   15])

In [22]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj jiang xxmaj xian uses the complex backstory of xxmaj ling xxmaj ling and xxmaj mao xxmaj daobing to'

To a Gpu, the array of tokens need to be split into batches of some size and fed in sequence with the model preserving the state of previous words to predict what comes next


In [23]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."

In [24]:
tokens = tkn(stream)

In [25]:
len(tokens)

90

In [26]:
#assume bs =6 of length 15
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len : (i+1)*seq_len] for i in range(bs)])

In [27]:
len(d_tokens), len(d_tokens[0])

(6, 15)

In [28]:
df = pd.DataFrame(d_tokens)

In [29]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
1,movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
2,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
3,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
4,of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we


In [30]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
len(d_tokens),len(d_tokens[0])

(6, 5)

In [31]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)


In [32]:
nums200 = toks200.map(num)

LMDataLoader is a useful data loader for language text by fastai

In [33]:
dls = LMDataLoader(nums200)

In [34]:
x,y = first(dls)

In [35]:
x.shape,y.shape
x[0].shape

torch.Size([72])

In [36]:
' '.join(num.vocab[o] for o in x[0])

'xxbos xxmaj jiang xxmaj xian uses the complex backstory of xxmaj ling xxmaj ling and xxmaj mao xxmaj daobing to study xxmaj mao \'s " cultural revolution " ( 1966 - 1976 ) at the village level . xxmaj the film has the elements and pace of xxmaj chinese opera and so appears slow and sometimes sentimental to the foreign viewer . xxmaj but the movie provides a window onto contemporary life'

In [37]:
' '.join(num.vocab[o] for o in y[0])

'xxmaj jiang xxmaj xian uses the complex backstory of xxmaj ling xxmaj ling and xxmaj mao xxmaj daobing to study xxmaj mao \'s " cultural revolution " ( 1966 - 1976 ) at the village level . xxmaj the film has the elements and pace of xxmaj chinese opera and so appears slow and sometimes sentimental to the foreign viewer . xxmaj but the movie provides a window onto contemporary life in'

for the first batch, x the independent variable contains words from 0 to 71 and y from 1 to 72

To pass the preprocessed text into a language model, we use a neural network pretrained on wikipedia corpus and then finetuned to work with imdb reviews. fastai dataloader class makes use of textloader class to load the data in the desired format.

In [38]:
get_imdb = partial(get_text_files,folders=['train','test','unsup'])

dls_lm = DataBlock(blocks=TextBlock.from_folder(path,is_lm=True), get_items=get_imdb,splitter=RandomSplitter(0.1)).dataloaders(path,path=path,bs=128,sq_len=80)

In [39]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj if you want your vision of xxmaj chaplin limited to a lovable tramp and you get your belly laughs from pathos , watch something else . xxmaj if , however , you love slapstick comedy as performed by one of the best , do watch this one . \n\n xxmaj the image is of the tramp who really can not get the girl . xxmaj he spots another couple kissing","xxmaj if you want your vision of xxmaj chaplin limited to a lovable tramp and you get your belly laughs from pathos , watch something else . xxmaj if , however , you love slapstick comedy as performed by one of the best , do watch this one . \n\n xxmaj the image is of the tramp who really can not get the girl . xxmaj he spots another couple kissing on"
1,"precisely because they were way to risky and would result in more losses from credit that is never repaid than profits from late fees interest . xxmaj but once the xxmaj federal xxmaj reserve drops the interest rates down to xxunk low levels , the risk changes and even those people who may not be ultimately able to pay beck their debt , can become profitable . xxmaj but xxmaj god forbid","because they were way to risky and would result in more losses from credit that is never repaid than profits from late fees interest . xxmaj but once the xxmaj federal xxmaj reserve drops the interest rates down to xxunk low levels , the risk changes and even those people who may not be ultimately able to pay beck their debt , can become profitable . xxmaj but xxmaj god forbid anyone"


To convert the word indices in the numericalarized dataset, we use activation function of embedding matrix similar to used in collaborative filtering and tabular data . this way, it becomes fit to be used in a recurrent neural network (AWD-LSTM) - the nut and bolts of this will be discussed further

Perplexity() refers to exp(crossentropy) ->  we know that crossentorpy loss refers to -log(softmax(x) where targ == 1. Perpelexity is often used in NLP models.

In [40]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3,metrics = [accuracy,Perplexity()]).to_fp16()

The AWD-LSTM is pre-trained model trained on wikipedia corpus of texts. Now, it is to be fine-tuned for IMDB reviews vocab - modify the embedding for the new words not found in wikipedia corpus-> only the non-available embeddings are to be trained now.so, others are frozen. IT takes lot of time to train, so the intermediate results after each epoch are stored. fine_tune method does not do it , so fit_one_cycle is used.

In [41]:
learn.fit_one_cycle(1,2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.005187,3.903093,0.300468,49.555504,41:10


30 percent accuracy after just one cycle of training -> now, we got to modify the network to do classification of movies based on imdb review -> the final layer needs to be modified. NLP models take longer time to train - because the independent and dependent are both long - 72 worded sequence for a batch size of 64

In [42]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [43]:
#after initial training with initial layers frozen, we could unfreeze and now fine tune for more epochs

In [44]:
learn.unfreeze()
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.738733,3.728958,0.320548,41.635708,34:26
1,3.598147,3.621960,0.332818,37.410828,30:46
2,3.484455,3.592586,0.336856,36.327908,31:09


Encoder : all the layers before the final activation layer which produces the output is often called in NLP related problems as Encoder. We save the entire model till the last layer ,i.e save the encoder.

In [2]:
learn.save_encoder('finetuned_encoder')

#Training the learner again and again is a very computationally expensive and time consuming process.
#the notebook stops here. The remaining is just fine-tuning the model for classification of reviews.

NameError: name 'learn' is not defined